In [19]:
import torch
import torchvision
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms 
from torch.utils.data import Dataset, DataLoader
import torch.nn.init
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

import matplotlib.pyplot as plt
import time

In [20]:
#하드웨어 설정

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777) 

In [21]:
torch.manual_seed(777)
torch.cuda.manual_seed(777)
torch.backends.cudnn.enabled = False 

In [22]:
# 파라미터 설정

learning_rate = 0.001
training_epochs = 50
batch_size = 200

In [23]:
transforms = transforms.Compose([
    transforms.Resize((35, 35)),
    transforms.ToTensor()
])

mnist_train = dsets.FashionMNIST(root='FashionMNIST_data/',
                            train=True,
                            transform=transforms,
                            download=True)
minist_test = dsets.FashionMNIST(root='FashionMNIST_data/',
                            train=False,
                            transform=transforms,
                            download=True)

100%|██████████| 26.4M/26.4M [00:07<00:00, 3.47MB/s]


Extracting FashionMNIST_data/FashionMNIST\raw\train-images-idx3-ubyte.gz to FashionMNIST_data/FashionMNIST\raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 114kB/s]


Extracting FashionMNIST_data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to FashionMNIST_data/FashionMNIST\raw



100%|██████████| 4.42M/4.42M [00:02<00:00, 1.98MB/s]


Extracting FashionMNIST_data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to FashionMNIST_data/FashionMNIST\raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 2.58MB/s]

Extracting FashionMNIST_data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to FashionMNIST_data/FashionMNIST\raw



In [24]:
train_loader = DataLoader(dataset=mnist_train,
                          batch_size=batch_size,
                            shuffle=True,
                            drop_last=True)
test_loader = DataLoader(dataset=minist_test,
                            batch_size=batch_size,
                                shuffle=False,
                                drop_last=True)


In [32]:
# 모델 생성

class Lenet5(torch.nn.Module):
  def __init__(self):
    super(Lenet5,self).__init__()

    self.l1=torch.nn.Conv2d(1,6,kernel_size=5,padding=0,stride=1)
    self.x1=torch.nn.Tanh()
    self.l2=torch.nn.AvgPool2d(kernel_size=2,padding=0,stride=2)

    self.l3=torch.nn.Conv2d(6,16,kernel_size=5,padding=0,stride=1)
    self.x2=torch.nn.Tanh()
    self.l4=torch.nn.AvgPool2d(kernel_size=2,padding=0,stride=2)

    self.l5=torch.nn.Flatten()

    self.l6=torch.nn.Linear(16*5*5,120,bias=True)
    self.x3=torch.nn.Tanh()

    self.l7=torch.nn.Linear(120,84,bias=True)
    self.x4=torch.nn.Tanh()

    self.l8=torch.nn.Linear(84,10,bias=True)
  


  def forward(self,x):
    out=self.l1(x)
    out=self.x1(out)
    out=self.l2(out)
    out=self.l3(out)
    out=self.x2(out)
    out=self.l4(out)

    out=out.view(out.size(0),-1)

    out=self.l6(out)
    out=self.x3(out)
    out=self.l7(out)
    out=self.x4(out)
    out=self.l8(out)
    return out

In [33]:
model = Lenet5().to(device) 

In [36]:
# 모델 최적화

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [38]:
# 모델 학습
total_batch = len(train_loader)
print('Learning started. It takes some time.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print(f'Epoch: {epoch + 1}, Cost: {avg_cost.item():.4f}')

print('Learning finished.')

Learning started. It takes some time.
Epoch: 1, Cost: 0.3261
Epoch: 2, Cost: 0.3127
Epoch: 3, Cost: 0.3023
Epoch: 4, Cost: 0.2938
Epoch: 5, Cost: 0.2852
Epoch: 6, Cost: 0.2760
Epoch: 7, Cost: 0.2699
Epoch: 8, Cost: 0.2653
Epoch: 9, Cost: 0.2582
Epoch: 10, Cost: 0.2500
Epoch: 11, Cost: 0.2437
Epoch: 12, Cost: 0.2390
Epoch: 13, Cost: 0.2324
Epoch: 14, Cost: 0.2278
Epoch: 15, Cost: 0.2222
Epoch: 16, Cost: 0.2186
Epoch: 17, Cost: 0.2127
Epoch: 18, Cost: 0.2069
Epoch: 19, Cost: 0.2019
Epoch: 20, Cost: 0.1979
Epoch: 21, Cost: 0.1931
Epoch: 22, Cost: 0.1870
Epoch: 23, Cost: 0.1841
Epoch: 24, Cost: 0.1804
Epoch: 25, Cost: 0.1750
Epoch: 26, Cost: 0.1708
Epoch: 27, Cost: 0.1673
Epoch: 28, Cost: 0.1648
Epoch: 29, Cost: 0.1587
Epoch: 30, Cost: 0.1542
Epoch: 31, Cost: 0.1514
Epoch: 32, Cost: 0.1461
Epoch: 33, Cost: 0.1454
Epoch: 34, Cost: 0.1401
Epoch: 35, Cost: 0.1352
Epoch: 36, Cost: 0.1312
Epoch: 37, Cost: 0.1272
Epoch: 38, Cost: 0.1260
Epoch: 39, Cost: 0.1223
Epoch: 40, Cost: 0.1194
Epoch: 41, 

In [39]:
# 모델 테스트

accuracy = 0
total_batch = len(test_loader)

with torch.no_grad(): # 학습을 진행하지 않을 것이므로 torch.no_grad()를 사용
    model.eval() # 모델을 평가 상태로 지정

In [41]:
for X, Y in test_loader:
    X = X.to(device)
    Y = Y.to(device)

    prediction = model(X)
    correct_prediction = torch.argmax(prediction, 1) == Y


In [42]:
x = torch.rand(3,3)
max_x = torch.max(x)
max_x_index = torch.argmax(x)

In [43]:
accuracy = correct_prediction.float().mean()

In [44]:
print('accuracy:', accuracy.item()/total_batch)

accuracy: 0.01759999990463257


In [45]:
x = torch.rand(1)
print(x)
print(x.item())

tensor([0.6163])
0.6163243651390076
